<a href="https://colab.research.google.com/github/S-Kaito/s-kaito.github.io/blob/master/notebook/security/2020_0618.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np

# 4行6列8bitのサイズ
SIZE = (6, 4, 8)

def xor(a, b):
    return int((a == 1) ^ (b == 1))  

# 2進数配列を16進数にする
def parse_hex(text):
    state = np.zeros((SIZE[0], SIZE[1]), dtype=int)

    for i in range(SIZE[0]):
        for j in range(SIZE[1]):
            bin_str = ""
            for k in range(SIZE[2] - 1, -1, -1):
                bin_str += str(text[i][j][k])
            state[i][j] = int(bin_str, 2)
    return state

# 16進数を2進数配列にする
def parse_bin(text):
    state = np.zeros(SIZE, dtype=int)

    for i in range(SIZE[0]):
        for j in range(SIZE[1]):
            bin_str = bin(text[i][j])
            for k in range(2, len(bin_str)):
                state[i][j][9 - k] = 1 if bin_str[k] == "1" else 0
    return state

# AddRoundKeyによる暗号化・復号
def add_round_key(text, key):
    state = np.zeros(SIZE, dtype=int)

    # 各bitとキーの排他的論理和を取る
    for i in range(SIZE[0]):
        for j in range(SIZE[1]):
            for k in range(SIZE[2]):
                state[i][j][k] = xor(text[i][j][k], key[i][j][k])

    return state

# ByteBoxによる暗号化
def byte_sub(text):
    state = np.zeros(SIZE, dtype=int)

    # 逆元を格納したテーブル
    sbox = [
        0x63,  0x7c,  0x77,  0x7b,  0xf2,  0x6b,  0x6f,  0xc5,  0x30,  0x01,  0x67,  0x2b,  0xfe,  0xd7,  0xab,  0x76,
        0xca,  0x82,  0xc9,  0x7d,  0xfa,  0x59,  0x47,  0xf0,  0xad,  0xd4,  0xa2,  0xaf,  0x9c,  0xa4,  0x72,  0xc0,
        0xb7,  0xfd,  0x93,  0x26,  0x36,  0x3f,  0xf7,  0xcc,  0x34,  0xa5,  0xe5,  0xf1,  0x71,  0xd8,  0x31,  0x15,
        0x04,  0xc7,  0x23,  0xc3,  0x18,  0x96,  0x05,  0x9a,  0x07,  0x12,  0x80,  0xe2,  0xeb,  0x27,  0xb2,  0x75,
        0x09,  0x83,  0x2c,  0x1a,  0x1b,  0x6e,  0x5a,  0xa0,  0x52,  0x3b,  0xd6,  0xb3,  0x29,  0xe3,  0x2f,  0x84,
        0x53,  0xd1,  0x00,  0xed,  0x20,  0xfc,  0xb1,  0x5b,  0x6a,  0xcb,  0xbe,  0x39,  0x4a,  0x4c,  0x58,  0xcf,
        0xd0,  0xef,  0xaa,  0xfb,  0x43,  0x4d,  0x33,  0x85,  0x45,  0xf9,  0x02,  0x7f,  0x50,  0x3c,  0x9f,  0xa8,
        0x51,  0xa3,  0x40,  0x8f,  0x92,  0x9d,  0x38,  0xf5,  0xbc,  0xb6,  0xda,  0x21,  0x10,  0xff,  0xf3,  0xd2,
        0xcd,  0x0c,  0x13,  0xec,  0x5f,  0x97,  0x44,  0x17,  0xc4,  0xa7,  0x7e,  0x3d,  0x64,  0x5d,  0x19,  0x73,
        0x60,  0x81,  0x4f,  0xdc,  0x22,  0x2a,  0x90,  0x88,  0x46,  0xee,  0xb8,  0x14,  0xde,  0x5e,  0x0b,  0xdb,
        0xe0,  0x32,  0x3a,  0x0a,  0x49,  0x06,  0x24,  0x5c,  0xc2,  0xd3,  0xac,  0x62,  0x91,  0x95,  0xe4,  0x79,
        0xe7,  0xc8,  0x37,  0x6d,  0x8d,  0xd5,  0x4e,  0xa9,  0x6c,  0x56,  0xf4,  0xea,  0x65,  0x7a,  0xae,  0x08,
        0xba,  0x78,  0x25,  0x2e,  0x1c,  0xa6,  0xb4,  0xc6,  0xe8,  0xdd,  0x74,  0x1f,  0x4b,  0xbd,  0x8b,  0x8a,
        0x70,  0x3e,  0xb5,  0x66,  0x48,  0x03,  0xf6,  0x0e,  0x61,  0x35,  0x57,  0xb9,  0x86,  0xc1,  0x1d,  0x9e,
        0xe1,  0xf8,  0x98,  0x11,  0x69,  0xd9,  0x8e,  0x94,  0x9b,  0x1e,  0x87,  0xe9,  0xce,  0x55,  0x28,  0xdf,
        0x8c,  0xa1,  0x89,  0x0d,  0xbf,  0xe6,  0x42,  0x68,  0x41,  0x99,  0x2d,  0x0f,  0xb0,  0x54,  0xbb,  0x16
    ]

    for i in range(SIZE[0]):
        for j in range(SIZE[1]):

            # 2進数配列の値を16進数にして、sboxから値を取得
            hex_value = sbox[sum([2 ** k * a for k, a in enumerate(key[i][j])])]
            # 16進数の値を2進数配列にする
            bin_value = bin(hex_value)
            block = np.zeros(8, dtype=int)
            for k, c in enumerate(bin_value[-1: 1: -1]):
                block[k] = 0 if c == '0' else 1

            # アフィン変換を行う
            block = np.dot([[1, 0, 0, 0, 1, 1, 1, 1], 
                            [1, 1, 0, 0, 0, 1, 1, 1], 
                            [1, 1, 1, 0, 0, 0, 1, 1], 
                            [1, 1, 1, 1, 0, 0, 0, 1], 
                            [1, 1, 1, 1, 1, 0, 0, 0], 
                            [0, 1, 1, 1, 1, 1, 0, 0], 
                            [0, 0, 1, 1, 1, 1, 1, 0], 
                            [0, 0, 0, 1, 1, 1, 1, 1]], np.array(block).T.copy())
            block = block + np.array([1, 1, 0, 0, 0, 1, 1, 0])
            block = [b % 2 for b in block]
            state[i][j] = block
    return state

# ByteSubの復号
def inv_byte_sub(text):
    state = np.zeros(SIZE, dtype=int)
    # 逆元を格納したテーブル
    sbox = [
        0x52, 0x09, 0x6a, 0xd5, 0x30, 0x36, 0xa5, 0x38, 0xbf, 0x40, 0xa3, 0x9e, 0x81, 0xf3, 0xd7, 0xfb,
        0x7c, 0xe3, 0x39, 0x82, 0x9b, 0x2f, 0xff, 0x87, 0x34, 0x8e, 0x43, 0x44, 0xc4, 0xde, 0xe9, 0xcb,
        0x54, 0x7b, 0x94, 0x32, 0xa6, 0xc2, 0x23, 0x3d, 0xee, 0x4c, 0x95, 0x0b, 0x42, 0xfa, 0xc3, 0x4e,
        0x08, 0x2e, 0xa1, 0x66, 0x28, 0xd9, 0x24, 0xb2, 0x76, 0x5b, 0xa2, 0x49, 0x6d, 0x8b, 0xd1, 0x25,
        0x72, 0xf8, 0xf6, 0x64, 0x86, 0x68, 0x98, 0x16, 0xd4, 0xa4, 0x5c, 0xcc, 0x5d, 0x65, 0xb6, 0x92,
        0x6c, 0x70, 0x48, 0x50, 0xfd, 0xed, 0xb9, 0xda, 0x5e, 0x15, 0x46, 0x57, 0xa7, 0x8d, 0x9d, 0x84,
        0x90, 0xd8, 0xab, 0x00, 0x8c, 0xbc, 0xd3, 0x0a, 0xf7, 0xe4, 0x58, 0x05, 0xb8, 0xb3, 0x45, 0x06,
        0xd0, 0x2c, 0x1e, 0x8f, 0xca, 0x3f, 0x0f, 0x02, 0xc1, 0xaf, 0xbd, 0x03, 0x01, 0x13, 0x8a, 0x6b,
        0x3a, 0x91, 0x11, 0x41, 0x4f, 0x67, 0xdc, 0xea, 0x97, 0xf2, 0xcf, 0xce, 0xf0, 0xb4, 0xe6, 0x73,
        0x96, 0xac, 0x74, 0x22, 0xe7, 0xad, 0x35, 0x85, 0xe2, 0xf9, 0x37, 0xe8, 0x1c, 0x75, 0xdf, 0x6e,
        0x47, 0xf1, 0x1a, 0x71, 0x1d, 0x29, 0xc5, 0x89, 0x6f, 0xb7, 0x62, 0x0e, 0xaa, 0x18, 0xbe, 0x1b,
        0xfc, 0x56, 0x3e, 0x4b, 0xc6, 0xd2, 0x79, 0x20, 0x9a, 0xdb, 0xc0, 0xfe, 0x78, 0xcd, 0x5a, 0xf4,
        0x1f, 0xdd, 0xa8, 0x33, 0x88, 0x07, 0xc7, 0x31, 0xb1, 0x12, 0x10, 0x59, 0x27, 0x80, 0xec, 0x5f,
        0x60, 0x51, 0x7f, 0xa9, 0x19, 0xb5, 0x4a, 0x0d, 0x2d, 0xe5, 0x7a, 0x9f, 0x93, 0xc9, 0x9c, 0xef,
        0xa0, 0xe0, 0x3b, 0x4d, 0xae, 0x2a, 0xf5, 0xb0, 0xc8, 0xeb, 0xbb, 0x3c, 0x83, 0x53, 0x99, 0x61,
        0x17, 0x2b, 0x04, 0x7e, 0xba, 0x77, 0xd6, 0x26, 0xe1, 0x69, 0x14, 0x63, 0x55, 0x21, 0x0c, 0x7d,
    ];

    for i in range(SIZE[0]):
        for j in range(SIZE[1]):
                     
            # アフィン変換を行う
            block = np.dot([[0, 1, 0, 1, 0, 0, 1, 0], 
                            [0, 0, 1, 0, 1, 0, 0, 1], 
                            [1, 0, 0, 1, 0, 1, 0, 0], 
                            [0, 1, 0, 0, 1, 0, 1, 0], 
                            [0, 0, 1, 0, 0, 1, 0, 1], 
                            [1, 0, 0, 1, 0, 0, 1, 0], 
                            [0, 1, 0, 0, 1, 0, 0, 1], 
                            [1, 0, 1, 0, 0, 1, 0, 0]], text[i][j].T.copy())
            block = block + np.array([0, 0, 0, 0, 0, 1, 0, 1])
            block = [b % 2 for b in block]

            # 2進数配列の値を16進数にして、sboxから値を取得
            hex_value = sbox[sum([2 ** k * a for k, a in enumerate(block)])]
            # 16進数の値を2進数配列にする
            bin_value = bin(hex_value)
            for k, c in enumerate(bin_value[-1: 1: -1]):
                block[k] = 0 if c == '0' else 1
            state[i][j] = block
            
    return state

# ShiftRowによる暗号化
def shift_row(text):
    state = np.zeros(SIZE, dtype=int)

    # 0行目を0こ左にずらす
    state[:, 0] = text[:, 0]
    # 1行目を1こ左にずらす
    state[:, 1] = np.append(text[1:, 1], np.array([text[0][1]]), axis=0)
    # 2行目を2こ左にずらす
    state[:, 2] = np.append(text[2:, 2], text[:2, 2], axis=0)
    # 3行目を3こ左にずらす
    state[:, 3] = np.append(text[3:, 3], text[:3, 3], axis=0)

    return state

# ShiftRowの復号
def inv_shift_row(text):
    state = np.zeros(SIZE, dtype=int)

    print(np.append(text[1:, 1], np.array([text[0][1]]), axis=0))
    # 0行目を0こ右にずらす
    state[:, 0] = text[:, 0]
    # 1行目を1こ右にずらす
    state[:, 1] = np.append(np.array([text[-1][1]]), text[0:-1, 1], axis=0)
    # 2行目を2こ右にずらす
    state[:, 2] = np.append(text[-2:, 2], text[:-2, 2], axis=0)
    # 3行目を3こ右にずらす
    state[:, 3] = np.append(text[-3:, 3], text[:-3, 3], axis=0)

    return state

# 剰余を含めた掛け算
def xtime(h):
    return (h << 1) ^ (0x1b if (len(bin(h)) == 10) else 0x00)

# 任意の要素同士の掛け算
def dot(x, y):
    mask = 1
    product = 0

    # 下位bitから比較してxorを行っていく
    for _ in range(len(bin(y)) - 2):
        if bin(y)[mask * -1] == '1':
            product ^= x
        x = xtime(x)
        mask += 1
    return product

# MixColumnによる暗号化
def mix_column(text):
    state = parse_hex(text)

    for c in range(6):

        # 行列の掛け算
        t0 = dot(2, state[c][0]) ^ dot(3, state[c][1]) ^        state[c][2]  ^        state[c][3]
        t1 =        state[c][0]  ^ dot(2, state[c][1]) ^ dot(3, state[c][2]) ^        state[c][3]
        t2 =        state[c][0]  ^        state[c][1]  ^ dot(2, state[c][2]) ^ dot(3, state[c][3])
        t3 = dot(3, state[c][0]) ^        state[c][1]  ^        state[c][2]  ^ dot(2, state[c][3])
        state[c][0] = t0
        state[c][1] = t1
        state[c][2] = t2
        state[c][3] = t3

    return parse_bin(state)

# MixColumnの復号
def inv_mix_column(text):
    state = parse_hex(text)

    for c in range(6):

        # 行列の掛け算
        t0 = dot(0x0e, state[c][0]) ^ dot(0x0b, state[c][1]) ^ dot(0x0d, state[c][2]) ^ dot(0x09, state[c][3])
        t1 = dot(0x09, state[c][0]) ^ dot(0x0e, state[c][1]) ^ dot(0x0b, state[c][2]) ^ dot(0x0d, state[c][3])
        t2 = dot(0x0d, state[c][0]) ^ dot(0x09, state[c][1]) ^ dot(0x0e, state[c][2]) ^ dot(0x0b, state[c][3])
        t3 = dot(0x0b, state[c][0]) ^ dot(0x0d, state[c][1]) ^ dot(0x09, state[c][2]) ^ dot(0x0e, state[c][3])
        state[c][0] = t0
        state[c][1] = t1
        state[c][2] = t2
        state[c][3] = t3

    return parse_bin(state)

In [ ]:
plain = np.random.randint(0, 2, SIZE) # 平分
key = np.random.randint(0, 2, SIZE) # 鍵

print("Plain")
print(plain)

# AddRoundKeyを行う
plain = add_round_key(plain, key)
print("AddRoundKey")
print(plain)
if np.all(plain == add_round_key(add_round_key(plain, key), key)):
    # 暗号化して復号したものと元のデータが一致するか
    print("Add Round key is complete")

# ByteSubを行う
plain = byte_sub(plain)
print("ByteSub")
print(plain)
if np.all(plain == inv_byte_sub(byte_sub(plain))):
    # 暗号化して復号したものと元のデータが一致するか
    print("Byte Sub is complete")

# ShiftRowを行う
plain = shift_row(plain)
print("ShiftRow")
print(plain)
if np.all(plain == inv_shift_row(shift_row(plain))):
    # 暗号化して復号したものと元のデータが一致するか
    print("Shift Row is complete")

# MixColumnを行う
cipher = mix_column(plain)
print("MixColumn")
print(cipher)
if np.all(cipher == inv_mix_column(mix_column(cipher))):
    # 暗号化して復号したものと元のデータが一致するか
    # 本来ならば表示される
    print("Mix column is complete")

Plain
[[[0 1 1 0 0 0 1 1]
  [0 1 1 0 0 0 1 0]
  [0 1 0 1 1 1 1 0]
  [1 0 1 1 1 0 0 1]]

 [[1 1 0 1 0 0 0 0]
  [1 0 0 0 1 0 1 0]
  [0 1 1 1 1 0 1 0]
  [1 0 1 1 1 0 0 0]]

 [[1 0 1 1 0 1 1 1]
  [1 0 0 0 0 0 0 0]
  [0 0 1 1 0 1 0 1]
  [1 1 0 0 1 0 0 0]]

 [[1 1 0 0 0 1 1 1]
  [1 0 1 1 1 1 1 0]
  [1 0 0 1 1 0 0 1]
  [1 1 0 0 1 0 0 0]]

 [[1 0 1 1 1 0 1 1]
  [0 1 0 1 1 0 1 0]
  [1 0 0 1 0 1 0 0]
  [0 1 0 1 1 0 1 1]]

 [[1 1 1 1 1 0 1 0]
  [0 1 1 0 0 1 0 0]
  [0 0 1 1 1 0 0 1]
  [0 1 1 0 1 0 1 1]]]
AddRoundKey
[[[1 0 0 0 1 0 0 0]
  [1 1 0 1 1 1 0 0]
  [0 0 1 1 0 0 0 0]
  [0 1 1 0 1 1 0 0]]

 [[1 0 0 1 1 1 0 0]
  [0 1 0 1 0 0 0 1]
  [0 0 0 1 0 0 1 1]
  [1 1 0 1 1 0 1 1]]

 [[1 0 1 1 1 1 1 1]
  [1 1 0 1 0 0 0 1]
  [1 1 1 1 0 0 1 0]
  [0 0 0 1 1 1 1 0]]

 [[1 1 1 0 1 0 1 0]
  [0 1 1 1 0 0 0 0]
  [0 0 0 0 1 0 0 0]
  [1 0 1 1 1 1 0 0]]

 [[0 1 0 1 0 1 1 1]
  [1 1 1 1 1 1 0 1]
  [0 1 1 0 1 0 0 0]
  [1 1 0 1 0 0 0 1]]

 [[1 0 0 1 1 0 0 0]
  [0 1 1 1 1 0 1 1]
  [1 1 0 0 1 1 0 1]
  [1 0 0 0 0 0 1 0]]